In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os
import tqdm

# ===== Configuración =====

In [15]:
PATH = "../../../data/raw/VCoR/"
TEST_PATH = f"{PATH}/test"
TRAIN_PATH = f"{PATH}/train"
VAL_PATH = f"{PATH}/val"

BATCH_SIZE = 32
IMG_SIZE = 128
NUM_EPOCHS = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ===== Transforms =====

In [16]:
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)  # Normalización entre -1 y 1
])

In [17]:
train_dataset = datasets.ImageFolder(TRAIN_PATH, transform=transform)
val_dataset = datasets.ImageFolder(VAL_PATH, transform=transform)
test_dataset = datasets.ImageFolder(TEST_PATH, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# CNN MODEL

In [18]:
class CarCNN(nn.Module):
    def __init__(self, num_classes):
        super(CarCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * (IMG_SIZE // 8) * (IMG_SIZE // 8), 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [19]:
# Instanciar modelo
model = CarCNN(num_classes=len(train_dataset.classes)).to(DEVICE)

# Pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [20]:
# Entrenamiento
for epoch in tqdm.tqdm(range(NUM_EPOCHS)):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = correct / total
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} | Loss: {running_loss:.4f} | Train Acc: {train_acc:.4f}")
        
    # Validación
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_acc = correct / total
    print(f"Validation Acc: {val_acc:.4f}")


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1/10 | Loss: 280.0557 | Train Acc: 0.5780


 10%|█         | 1/10 [02:15<20:19, 135.46s/it]

Validation Acc: 0.7387
Epoch 2/10 | Loss: 176.4809 | Train Acc: 0.7322


 20%|██        | 2/10 [03:52<15:02, 112.87s/it]

...

Validation Acc: 0.7832
Epoch 9/10 | Loss: 101.4981 | Train Acc: 0.8376


 90%|█████████ | 9/10 [16:10<01:42, 102.90s/it]

Validation Acc: 0.8206
Epoch 10/10 | Loss: 95.0032 | Train Acc: 0.8506


100%|██████████| 10/10 [17:49<00:00, 106.98s/it]

Validation Acc: 0.8226


In [21]:
# Evaluación final en el test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

In [22]:
test_acc = correct / total
print(f"\n✅ Test Accuracy: {test_acc:.4f}")


✅ Test Accuracy: 0.8072


In [24]:
torch.save(model.state_dict(), "../trained/modelo_color.pt")